# 1. Load the dataset

In [1]:
from datasets import load_dataset
import torch

raw_datasets = load_dataset("conll2003", trust_remote_code=True)
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

# 2. List Labels for words available in the dataset

In [2]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

In [3]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

# 3. Showcase mapping between words and tokens

In [4]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


# 4. Load BERT tokenizer

In [5]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenizer.is_fast

True

# 5. Example of how to tokenize a sentence

In [6]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

Code above introduces an issue: word lamb was split into two, and additional special tokens were added. Now, we have an input of len 12, while having 9 labels. We need to align them.

In [7]:
inputs.word_ids() # Mapping between tokens and words

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

In [8]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [9]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


# 6. Function for both tokenization and alignment

In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

Use the function to tokenize and align labels for all datasets. map() function applies a function to all elements of a list,
and with batched = True we apply it to all elements of a batch to speed up the process.

In [11]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
tokenized_datasets

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

# 7. Data Collation
Now we need to collate the data into batches, and pad the sequences to the same length. This way, sentences in every batch will be the same length,
but the length will vary between batches.

In [12]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

As you can see, the labels are padded with -100, which is a special token that will be ignored by this model during training.

# 8. Define metrics for the model

In [13]:
import evaluate

metric = evaluate.load("seqeval")

# Take labels for the first example
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [14]:
# Make a prediction with a wrong label to showcase thew result of metrics computation
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': np.float64(1.0),
  'recall': np.float64(0.5),
  'f1': np.float64(0.6666666666666666),
  'number': np.int64(2)},
 'ORG': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(1)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(0.6666666666666666),
 'overall_f1': np.float64(0.8),
 'overall_accuracy': 0.8888888888888888}

# 9. Define proper metrics for the model

In [15]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

# 10. Define the model

In [16]:
# Define mapping between labels and ids
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

print(id2label)
print(label2id)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC', 7: 'B-MISC', 8: 'I-MISC'}
{'O': 0, 'B-PER': 1, 'I-PER': 2, 'B-ORG': 3, 'I-ORG': 4, 'B-LOC': 5, 'I-LOC': 6, 'B-MISC': 7, 'I-MISC': 8}


In [17]:
from transformers import AutoModelForTokenClassification

# Load the model
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)
model.config.num_labels

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


9

# 11 Fine-tune the model

In [18]:
# Log in to the Hugging Face Hub
from huggingface_hub import notebook_login

notebook_login()

In [22]:
from transformers import TrainingArguments

# Arguments for training
args = TrainingArguments(
    "bert-finetuned-ner",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

# 12. Train the model

In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train(); #On my PC this will take around 90 minutes, uncomment at your own risk

# Save the model to the Hugging Face Hub
trainer.push_to_hub(commit_message="Training complete")

C:\Users\gusta\AppData\Local\Temp\ipykernel_18612\2007957657.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.077900,0.069629,0.899496,0.930831,0.914895,0.981662
2,0.034500,0.070156,0.936979,0.945809,0.941374,0.984959
3,0.022000,0.064164,0.936724,0.951700,0.944152,0.986519


model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/GustawB/bert-finetuned-ner/commit/e10a0c20bc2c293205ee90a5c9ec49513266e693', commit_message='Training complete', commit_description='', oid='e10a0c20bc2c293205ee90a5c9ec49513266e693', pr_url=None, repo_url=RepoUrl('https://huggingface.co/GustawB/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='GustawB/bert-finetuned-ner'), pr_revision=None, pr_num=None)

# 13.5 Fine tune other models for comparison

In [26]:
# Fine-tune DistilBert

distilBertModel = AutoModelForTokenClassification.from_pretrained(
    "distilbert-base-cased",
    id2label=id2label,
    label2id=label2id,
)

distilBertAgrs = TrainingArguments(
    "distilbert-finetuned-ner-gbgb",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

distilBerttrainer = Trainer(
    model=distilBertModel,
    args=distilBertAgrs,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
distilBerttrainer.train()
distilBerttrainer.push_to_hub(commit_message="Training complete")

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\gusta\AppData\Local\Temp\ipykernel_18612\585876913.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  distilBerttrainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.093700,0.089384,0.873695,0.901043,0.887158,0.973303
2,0.045000,0.083326,0.911667,0.925783,0.918671,0.980176
3,0.029900,0.074842,0.913895,0.934197,0.923935,0.982016


HfHubHTTPError: (Request ID: Root=1-6729eb02-7e12797b7e96f9ee4f0330e0;95de3382-7784-4d02-bf77-6b4f5ccf5933)

403 Forbidden: You don't have the rights to create a model under the namespace "GustawB".
Cannot access content at: https://huggingface.co/api/repos/create.
Make sure your token has the correct permissions.

In [30]:
# Fine-tune AlBERT

AlBertModel = AutoModelForTokenClassification.from_pretrained(
    "albert-base-v2",
    id2label=id2label,
    label2id=label2id,
)

AlBertAgrs = TrainingArguments(
    "albert-finetuned-ner-gbgb",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

AlBerttrainer = Trainer(
    model=AlBertModel,
    args=AlBertAgrs,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
AlBerttrainer.train()
AlBerttrainer.push_to_hub(commit_message="Training complete")

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\gusta\AppData\Local\Temp\ipykernel_18612\2233204810.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  AlBerttrainer = Trainer(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.537900,0.484265,0.407918,0.273982,0.327796,0.850209
2,0.349100,0.372616,0.490323,0.383709,0.430514,0.877759
3,0.260000,0.337130,0.503215,0.460956,0.481159,0.889813


model.safetensors:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/GustawB/albert-finetuned-ner-gbgb/commit/80561fd086b38182d566b91377f5ef1fc8d20e58', commit_message='Training complete', commit_description='', oid='80561fd086b38182d566b91377f5ef1fc8d20e58', pr_url=None, repo_url=RepoUrl('https://huggingface.co/GustawB/albert-finetuned-ner-gbgb', endpoint='https://huggingface.co', repo_type='model', repo_id='GustawB/albert-finetuned-ner-gbgb'), pr_revision=None, pr_num=None)

# 13 Use the model

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint, I use this so that training won't kill my PC
model_checkpoint = "huggingface-course/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("My name is Sylvain and I work at Hugging Face in Brooklyn.")

# 14 Playing with bert-base-cased model
As you probably noticed, we are fine-tuning bert-base-cased model. You can find it here: https://huggingface.co/google-bert/bert-base-cased?text=The+goal+of+life+is+%5BMASK%5D.
According to its description, its main purpose is to be fine-tuned on tasks that use the whole sentence (potentially masked) to make decisions.
Still, it supports token classification (showcased above), but also sentence prediction (example below).

In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-cased')
unmasker("Hello I'm a [MASK] model.")

# 15. Models to train and test

https://huggingface.co/FacebookAI/xlm-roberta-base \
https://huggingface.co/distilbert/distilroberta-base \
https://huggingface.co/google-bert/bert-base-cased \
https://huggingface.co/google-bert/bert-base-uncased \
https://huggingface.co/distilbert/distilbert-base-cased \
https://huggingface.co/distilbert/distilbert-base-uncased \
https://huggingface.co/microsoft/deberta-v3-base \
https://huggingface.co/albert/albert-base-v2 \
https://huggingface.co/huawei-noah/TinyBERT_General_4L_312D (Very lacking docs) \
https://huggingface.co/prajjwal1/bert-tiny (some dude) \